In [8]:
from pySWATPlus import TxtinoutReader
import random
from SALib.sample import sobol as sobol_sample
from SALib.analyze import sobol as sobol_analyze
import numpy as np
import random
import concurrent.futures

In [9]:
random.seed(42)  # Set seed for reproducibility

In [10]:
txtinout_folder = '/mnt/c/users/joans/onedrive/Escriptori/icra/muga_windows'    #Here you have to put the path to your  txtinout folder, with the SWAT.exe 
txtinout_reader = TxtinoutReader(txtinout_folder)

In [11]:
txtinout_reader.set_beginning_and_end_year(2010, 2012)
txtinout_reader.set_warmup(1)
txtinout_reader.enable_object_in_print_prt('channel_sd', True, False, False, False)

In [12]:
#define the problem: in our case, we just want to calibrate epco and esco and evaluate the model

def run_and_evaluate_swat(epco: int = 0.5, esco: int = 0.5):
    
    print(f'Running SWAT with epco = {epco} and esco = {esco} \n')
    
    #copy swat so simulations don't interfere with each other and can run in parallel
    tmp_path = txtinout_reader.copy_swat()
    
    reader = TxtinoutReader(tmp_path)
    hydrology_hyd = reader.register_file('hydrology.hyd', has_units = False, )
    hydrology_hyd_df = hydrology_hyd.df
    
    #overwrite the values of epco and esco
    hydrology_hyd_df['epco'] = epco
    hydrology_hyd_df['esco'] = esco
    
    #save the changes
    hydrology_hyd.overwrite_file()
    
    #run the model
    txt_in_out_result = reader.run_swat(show_output=False)
    
    #read the results
    reader = TxtinoutReader(txt_in_out_result)    
    channel_sdmorph = reader.register_file('channel_sdmorph_day.txt', has_units = True, )
    
    #here, you should read your observations and calculate the objective function
    #for now, let's just return a mock value
    return random.random()

# Wrapper function for parallel execution
def evaluate(params):
    return run_and_evaluate_swat(*params)

In [13]:
# Define the model inputs
problem = {
  'num_vars': 2,
  'names': ['epco', 'esco'],
  'bounds': [[0, 1]]*2
}

In [18]:
# sample
param_values = sobol_sample.sample(problem, 2)
# Parallel execution
with concurrent.futures.ProcessPoolExecutor() as executor:
    y = np.array(list(executor.map(evaluate, param_values)))



Running SWAT with epco = 0.4832058800384402 and esco = 0.9244963796809316 
Running SWAT with epco = 0.1337153371423483 and esco = 0.6604474615305662 
Running SWAT with epco = 0.1337153371423483 and esco = 0.9244963796809316 
Running SWAT with epco = 0.6682199016213417 and esco = 0.2060658447444439 
Running SWAT with epco = 0.576692558825016 and esco = 0.2060658447444439 
Running SWAT with epco = 0.4832058800384402 and esco = 0.9244963796809316 

Running SWAT with epco = 0.6682199016213417 and esco = 0.0987796438857913 
Running SWAT with epco = 0.6682199016213417 and esco = 0.0987796438857913 
Running SWAT with epco = 0.576692558825016 and esco = 0.2060658447444439 
Running SWAT with epco = 0.4832058800384402 and esco = 0.6604474615305662 
Running SWAT with epco = 0.576692558825016 and esco = 0.0987796438857913 





Running SWAT with epco = 0.1337153371423483 and esco = 0.6604474615305662 








In [19]:
sobol_indices = sobol_analyze.analyze(problem, y)
sobol_indices

{'S1': array([0.64496951, 0.81568241]),
 'S1_conf': array([0.78872912, 1.01510526]),
 'ST': array([2.87531541, 5.37690055]),
 'ST_conf': array([1.86227058, 4.79791637]),
 'S2': array([[        nan, -1.79707892],
        [        nan,         nan]]),
 'S2_conf': array([[       nan, 4.30014452],
        [       nan,        nan]])}